# Group assignment object detection

### Choice of model

In this project, the task is to determine the birds and their location inside a picture. Birds can be of many differnt shapes and forms, so it is important to have a generalizable model. Secondly, it is critical that the model can make predictions in a tractable manner. It should be fast enough to have the model running on inference on the drone's hardware. For that reason, the model can't be large and complex, it should be lightweight and apaptable. 

YOLO is a model that is known for its speed, it is fast and flexible. This will be the basis of our model's architecture. Yolo predicts a location bound for an object. 

An issue I have with the data is that I want to use apple's background removing feature but because of the file names. That are super long and complex as they are hashes, it gets confused because the hash potentially mimics a hash that is protected or another reason. So the best appraoch is to clean the file names. But I don't want to do that as it seems cumbersome as my whole code is already great as it is now. 

In [1]:
import sys
sys.path.append("/notebooks/DL---detection-of-birds-in-drone-images/data_augmentation")
import image_objects
from image_objects import AllImages

In [2]:


def get_all_files():
    all_images = AllImages()
    repo_path = "/notebooks/DL---detection-of-birds-in-drone-images"
    test_images = all_images.get_files_in_data_folder(f"{repo_path}/data/Harmful Birds Detection.v1i.yolov11/test")
    train_images = all_images.get_files_in_data_folder(f"{repo_path}/data/Harmful Birds Detection.v1i.yolov11/train")
    valid_images = all_images.get_files_in_data_folder(f"{repo_path}/data/Harmful Birds Detection.v1i.yolov11/valid")
    return all_images

all_images_objects = get_all_files()





### Create export of images of pigeons and crows

In [108]:
# Only needed to do this line once. After that you use apple shortcuts to process these images
#all_images_objects.copy_crows_pigeons("data/files_to_clean_nopixels")

1913


Exception: folder already exists

# Preprocess locally using Apple shortcuts

Now you made a folder with "crows" and "pigeons" as subfolders. You need to use apple shortcuts to remove the background of these pictures and add them to a folder. Do this seperately for the pigeons and crows folders. Important to create a "pigeons" and "crows" folder in this new folder. Use this new folder to add the pictures without background to the existing AllImages object

Link to Apple shortcut (Self made)
https://www.icloud.com/shortcuts/cb718c0257a8443fb68a9d5243597a47

### Recognized problem

Some of the images have instead of 4 yolo coordinates have 6 and many classifications, even though there is only a single pigeon in the bird. Therefore these are removed. These pictures are annoated with an outline around the bird, but that is not the correct format for our project

In [3]:
repo_path = "/notebooks/DL---detection-of-birds-in-drone-images"
all_images_objects.load_removed_background_pictures(f"{repo_path}/data/Subject images crows/Subjects not pixelated")

In [6]:
image = all_images_objects.get_random_cropped_images(25)



1777
img655_jpeg_jpg
/notebooks/DL---detection-of-birds-in-drone-images/data/Subject images crows/Subjects not pixelated/crows/img655_jpeg_jpg.rf Background Removed.47135de24d5a59f45f96e251d6909833.png


Opencv to add the pictures to the pictures